In [1]:
include("../src/Julia.jl")

objc[2476]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2476]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2476]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2476]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/se_u_5/"

"../../graphs/se_u_5/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  1.280201 seconds (1.27 M allocations: 80.224 MB, 3.75% gc time)
  0.446882 seconds (1.71 M allocations: 35.211 MB, 15.58% gc time)
 33.941505 seconds (321.56 M allocations: 8.091 GB, 4.70% gc time)
Working on iteration 10 with a-norm error 29774.86560618406
Working on iteration 20 with a-norm error 26005.22701244315
Working on iteration 30 with a-norm error 21391.94063071931
Working on iteration 40 with a-norm error 18663.48677945892
Working on iteration 50 with a-norm error 16579.620535884773
Working on iteration 60 with a-norm error 14468.653738911053
Working on iteration 70 with a-norm error 13006.059228821143
Working on iteration 80 with a-norm error 11730.200833107645
Working on iteration 90 with a-norm error 10850.959273881354
Working on iteration 100 with a-norm error 9598.02011885954
Working on iteration 110 with a-norm error 8646.336330453765
Working on iteration 120 with a-norm error 7755.066885861573
Working on iteration 130 with a-norm error 6816.529104606822
Working on i